In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import numpy as np
import pandas as pd
import torch.optim as opt
import sys
import utils
import hyp
from utils import PrioritizedReplayBuffer 
from rdkit import Chem
from rdkit.Chem import QED, AllChem, rdFingerprintGenerator, Crippen, Descriptors, MACCSkeys
from rdkit.Chem.Crippen import MolLogP
from rdkit.Contrib.SA_Score import sascorer
from environment import Molecule
from torch.utils.tensorboard import SummaryWriter
import os
import subprocess
from tqdm import tqdm
import pickle
import catboost

In [5]:
class NoisyLinear(nn.Module):
    """Noisy Linear Layer for exploration"""
    def __init__(self, in_features, out_features, std_init=0.4):
        super(NoisyLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.std_init = std_init
        
        self.weight_mu = nn.Parameter(torch.Tensor(out_features, in_features))
        self.weight_sigma = nn.Parameter(torch.Tensor(out_features, in_features))
        self.bias_mu = nn.Parameter(torch.Tensor(out_features))
        self.bias_sigma = nn.Parameter(torch.Tensor(out_features))
        
        self.register_buffer('weight_epsilon', torch.Tensor(out_features, in_features))
        self.register_buffer('bias_epsilon', torch.Tensor(out_features))
        
        self.reset_parameters()
        self.reset_noise()
    
    def reset_parameters(self):
        mu_range = 1 / np.sqrt(self.in_features)
        self.weight_mu.data.uniform_(-mu_range, mu_range)
        self.weight_sigma.data.fill_(self.std_init / np.sqrt(self.in_features))
        self.bias_mu.data.uniform_(-mu_range, mu_range)
        self.bias_sigma.data.fill_(self.std_init / np.sqrt(self.out_features))
    
    def reset_noise(self):
        epsilon_in = self.scale_noise(self.in_features)
        epsilon_out = self.scale_noise(self.out_features)
        
        self.weight_epsilon.copy_(epsilon_out.outer(epsilon_in))
        self.bias_epsilon.copy_(epsilon_out)
    
    def forward(self, x):
        if self.training:
            weight = self.weight_mu + self.weight_sigma * self.weight_epsilon
            bias = self.bias_mu + self.bias_sigma * self.bias_epsilon
            return F.linear(x, weight, bias)
        else:
            return F.linear(x, self.weight_mu, self.bias_mu)
    
    @staticmethod
    def scale_noise(size):
        x = torch.randn(size)
        return x.sign().mul_(x.abs().sqrt_())

class RainbowDQN(nn.Module):
    """Rainbow DQN Network with Dueling Architecture and Distributional RL"""
    def __init__(self, input_length, output_length, atoms=51, v_min=-10, v_max=10):
        super(RainbowDQN, self).__init__()
        self.atoms = atoms
        self.v_min = v_min
        self.v_max = v_max
        self.output_length = output_length
        
        # Feature extraction
        self.linear_1 = NoisyLinear(input_length, 1024)
        self.linear_2 = NoisyLinear(1024, 512)
        
        # Dueling streams
        self.value_stream = nn.Sequential(
            NoisyLinear(512, 128),
            nn.ReLU(),
            NoisyLinear(128, atoms)
        )
        
        self.advantage_stream = nn.Sequential(
            NoisyLinear(512, 128),
            nn.ReLU(),
            NoisyLinear(128, output_length * atoms)
        )
            
    def forward(self, x):
       
        x = F.relu(self.linear_1(x))
        x = F.relu(self.linear_2(x))
        
        value = self.value_stream(x).view(-1, 1, self.atoms)
        advantage = self.advantage_stream(x).view(-1, self.output_length, self.atoms)
        
        q_dist = value + advantage - advantage.mean(dim=1, keepdim=True)
        return F.softmax(q_dist, dim=2)
    
    def reset_noise(self):
        for module in self.modules():
            if isinstance(module, NoisyLinear):
                module.reset_noise()
    
    def get_q_values(self, x):
        with torch.no_grad():
            dist = self.forward(x)
            support = torch.linspace(self.v_min, self.v_max, self.atoms).to(x.device)
            q_values = (dist * support).sum(dim=2)
            return q_values


In [6]:

REPLAY_BUFFER_CAPACITY = hyp.replay_buffer_size

def predict_activity(smiles: str) -> dict:
    """
    Предсказывает pIC50 и IC50 для молекулы по SMILES-строке.
    
    Аргументы:
        smiles (str): SMILES-представление молекулы
        
    Возвращает:
        dict: Словарь с предсказаниями pIC50 и IC50
        или сообщение об ошибке
    """
    MODEL = None
    COLUMNS = None
    
    # Загрузка модели и списка колонок при первом вызове
    if MODEL is None:
        try:
            with open('./submodel/final_catboost_model.pkl', 'rb') as f:
                MODEL = pickle.load(f)
            with open('./submodel/descriptor_columns.pkl', 'rb') as f:
                COLUMNS = pickle.load(f)
        except Exception as e:
            return {"error": f"Ошибка загрузки модели: {str(e)}"}
    
    # Преобразование SMILES в молекулярный объект
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return {"error": "Невалидный SMILES"}
    
    try:
        # Вычисление обычных дескрипторов
        desc_calc = {name: func for name, func in Descriptors.descList}
        row = {}
        
        # Разделение колонок на обычные и MACCS
        regular_cols = [col for col in COLUMNS if not col.startswith('maccs_')]
        maccs_cols = [col for col in COLUMNS if col.startswith('maccs_')]
        
        # Вычисление химических дескрипторов
        for col in regular_cols:
            if col in desc_calc:
                row[col] = desc_calc[col](mol)
            else:
                return {"error": f"Неизвестный дескриптор: {col}"}
        
        # Генерация MACCS-фингерпринтов
        fp = MACCSkeys.GenMACCSKeys(mol)
        for col in maccs_cols:
            bit_idx = int(col.split('_')[1])
            row[col] = 1 if fp.GetBit(bit_idx) else 0
        

        # Создание DataFrame с сохранением порядка колонок
        input_data = pd.DataFrame([row], columns=COLUMNS)
        
        # Предсказание pIC50
        pIC50 = MODEL.predict(input_data)[0]# Здесь модель обучалась на pIC50
        
        # Конвертация в IC50 (в наномолях)
        IC50_M = 10 ** (-pIC50)  # в молях
        IC50_nM = IC50_M * 1e9   # в наномолях

        # Расчет AlogP
        alogp = MolLogP(mol)
        
        return {
            "pIC50": round(pIC50, 6),
            "IC50": IC50_nM,
            "AlogP": round(alogp, 6)
        }
        
    except Exception as e:
        return {"error": f"Ошибка предсказания: {str(e)}"}


def has_pains(mol):
    """Обнаружение PAINS субструктур"""

    pains = [
             # Реакционноспособные
            "C(Cl)(Cl)Cl", "C#N", "N=C=O",
            # PAINS
            "C1COC1", "C=CC=O", "SC(=S)NC",
            # Металлоиды
            "[As]", "[Hg]",
            # Гетероциклы
            "c1nocc1", "c1nsnc1"
        ]
    PAINS_PATTERNS = [Chem.MolFromSmiles(s) for s in pains]
    for pattern in PAINS_PATTERNS:
        if pattern and mol.HasSubstructMatch(pattern):
            return True
    return False

def calculate_bbbp_score(mol):
    """Эвристика для предсказания проницаемости через ГЭБ на основе свойств молекулы"""
    logp = Descriptors.MolLogP(mol)
    mw = Descriptors.MolWt(mol)
    hbd = Descriptors.NumHDonors(mol)
    hba = Descriptors.NumHAcceptors(mol)
    
    # Критерии BBB проницаемости (Clark's rules)
    bbbp = (
        (logp > 1.0) and 
        (logp < 5.0) and 
        (mw < 500) and 
        (hbd <= 3) and 
        (hba <= 8)
    )
    return float(bbbp)

def reward_function(smiles: str, discount_factor: float) -> float:
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return -10.0  # Штраф за невалидные молекулы
    
    # 1. Предсказание активности
    activity = predict_activity(smiles)
    
    
    
    # 2. Ключевые параметры
    ic50 = activity["IC50"]
    pIC50 = activity["pIC50"]
    qed = QED.qed(mol)
    logp = activity["AlogP"]
    sas_score = sascorer.calculateScore(mol)  # Синтезируемость (1-10, меньше = лучше)
    bbbp_score = calculate_bbbp_score(mol)    # BBB проницаемость (0/1)
    
    # 3. Компоненты награды
    # Активность: экспоненциальное усиление низкого IC50
    activity_reward = 10 / (1 + np.exp(0.5 * (ic50 - 100))) 
    
    # Drug-likeness: QED + Lipinski filters
    druglikeness = qed * 0.7 + 0.3 * float(
        Descriptors.MolWt(mol) < 500 and
        Descriptors.NumHDonors(mol) <= 5 and
        Descriptors.NumHAcceptors(mol) <= 10
    )
    
    # Штрафы
    sas_penalty = np.clip((sas_score - 4) / 6, 0, 1)  # Штраф за сложность синтеза
    tox_penalty = 0.0
    if has_pains(mol):  # Проверка PAINS
        tox_penalty = 1.0
    
    # 4. Итоговая награда
    reward = (
        + 5.0 * activity_reward        # Приоритет активности
        + 2.0 * druglikeness           # Качество лекарства
        + 1.5 * bbbp_score             # Критично для ЦНС
        - 3.0 * sas_penalty            # Штраф за сложный синтез
        - 4.0 * tox_penalty            # Жесткий штраф за токсичность
        + 0.5 * (1 / (1 + abs(logp - 3))) # Оптимальный LogP (2-4)
    )
    
    return float(reward * discount_factor)

class QEDRewardMolecule(Molecule):
    
    def __init__(self, discount_factor, **kwargs):
        
        super(QEDRewardMolecule, self).__init__(**kwargs)
        self.discount_factor = discount_factor

    def _reward(self):
        return reward_function(self._state, self.discount_factor)
        
    def _goal_reached(self):
        mol = Chem.MolFromSmiles(self._state)
        if mol is None:
            return 0.0

        activity = predict_activity(self._state)
        IC50 = float(activity['IC50'])
        sas_score = sascorer.calculateScore(mol)  # Синтезируемость (1-10, меньше = лучше)
        bbbp_score = calculate_bbbp_score(mol)    # BBB проницаемость (0/1)
        qed = QED.qed(mol)

        return (
        IC50 < 50 and 
        qed > 0.6 and 
        sas_score < 5 and 
        bbbp_score == 1 and 
        not has_pains(mol)
        )

In [7]:
 class RainbowAgent:
    def __init__(self, input_length, output_length, device, atoms=51, v_min=-10, v_max=10):
        self.device = device
        self.atoms = atoms
        self.v_min = v_min
        self.v_max = v_max
        self.delta_z = (v_max - v_min) / (atoms - 1)
        self.support = torch.linspace(v_min, v_max, atoms).to(device)
        
        # Main and target networks
        self.dqn = RainbowDQN(input_length, output_length, atoms, v_min, v_max).to(device)
        self.target_dqn = RainbowDQN(input_length, output_length, atoms, v_min, v_max).to(device)
        self.target_dqn.load_state_dict(self.dqn.state_dict())
        
        self.replay_buffer = PrioritizedReplayBuffer(hyp.replay_buffer_size)
        self.optimizer = getattr(opt, hyp.optimizer)(self.dqn.parameters(), lr=hyp.learning_rate)
        self.times_of_update = 0
    
    def get_action(self, observations, epsilon_threshold):
        if np.random.uniform() < epsilon_threshold:
            return np.random.randint(0, observations.shape[0])
        
        observations = observations.to(self.device)
        with torch.no_grad():
            q_values = self.dqn.get_q_values(observations).cpu()
        return torch.argmax(q_values).item()
    
    def update_params(self, batch_size, gamma, polyak):
        if len(self.replay_buffer) < batch_size:
            return None
        
        # Sample from prioritized replay buffer
        samples, indices, weights = self.replay_buffer.sample(batch_size)
        weights = weights.to(self.device)
        
        # Unpack batch
        states, _, rewards, next_states, dones = zip(*samples)
        states = torch.stack([torch.FloatTensor(s) for s in states]).to(self.device)
        next_states = torch.stack([torch.FloatTensor(ns) for ns in next_states]).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)
        
        # Distributional DQN update
        with torch.no_grad():
            # Next state distribution
            next_dist = self.target_dqn(next_states)
            next_q = (next_dist * self.support).sum(2)
            next_actions = next_q.argmax(1)
            
            # Project next distribution
            next_dist = next_dist[range(batch_size), next_actions]
            rewards = rewards.unsqueeze(1).expand_as(next_dist)
            dones = dones.unsqueeze(1).expand_as(next_dist)
            support = self.support.unsqueeze(0).expand_as(next_dist)
            
            Tz = rewards + gamma * support * (1 - dones)
            Tz = Tz.clamp(self.v_min, self.v_max)
            b = (Tz - self.v_min) / self.delta_z
            l = b.floor().long()
            u = b.ceil().long()
            
            offset = torch.linspace(0, (batch_size - 1) * self.atoms, batch_size).long()\
                .unsqueeze(1).expand(batch_size, self.atoms).to(self.device)
            
            proj_dist = torch.zeros(next_dist.size()).to(self.device)
            proj_dist.view(-1).index_add_(0, (l + offset).view(-1), 
                                          (next_dist * (u.float() - b)).view(-1))
            proj_dist.view(-1).index_add_(0, (u + offset).view(-1), 
                                          (next_dist * (b - l.float())).view(-1))

        
        # Current state distribution
        dist = self.dqn(states)
        actions = torch.argmax(self.dqn.get_q_values(states), dim=1)
        dist = dist[range(batch_size), actions]
        
        # Calculate loss
        log_dist = torch.log(dist.clamp(min=1e-5))
        loss = - (proj_dist * log_dist).sum(1)
        weighted_loss = (weights * loss).mean()
        
        # Backpropagation
        self.optimizer.zero_grad()
        weighted_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.dqn.parameters(), 10)
        self.optimizer.step()
        
        # Update priorities
        priorities = loss.detach().cpu().numpy() + 1e-5
        self.replay_buffer.update_priorities(indices, priorities)
        
        # Update target network
        if self.times_of_update % hyp.update_interval == 0:
            with torch.no_grad():
                for param, target_param in zip(self.dqn.parameters(), self.target_dqn.parameters()):
                    target_param.data.copy_(polyak * target_param.data + (1 - polyak) * param.data)
        
        self.times_of_update += 1
        self.dqn.reset_noise()
        self.target_dqn.reset_noise()
        
        return weighted_loss.item()

In [13]:
# Инициализация
TENSORBOARD_LOG = True
TB_LOG_PATH = "./runs/dqn/run2"
episodes = 0
iterations = 25000
num_updates_per_it = 1

initmols = pd.read_csv("./InitMols.csv", sep=';')["Smiles"].to_numpy()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
environment = QEDRewardMolecule(
    discount_factor=hyp.discount_factor,
    atom_types=set(hyp.atom_types),
    init_mols = initmols,
    allow_removal=hyp.allow_removal,
    allow_no_modification=hyp.allow_no_modification,
    allow_bonds_between_rings=hyp.allow_bonds_between_rings,
    allowed_ring_sizes=set(hyp.allowed_ring_sizes),
    max_steps=hyp.max_steps_per_episode,
)

# Rainbow Agent вместо DQN
agent = RainbowAgent(
    input_length=hyp.fingerprint_length + 1,
    output_length=1,
    device=device,
    atoms=51,
    v_min=-10,
    v_max=10
)

# Загрузка весов
# agent.dqn.load_state_dict(torch.load("best_weights.pt"))

if TENSORBOARD_LOG:
    writer = SummaryWriter(TB_LOG_PATH)

environment.initialize()
eps_threshold = 1
best_reward = -1000

In [15]:
eps_threshold = 0.6
agent.dqn.load_state_dict(torch.load('best_weights.pt', weights_only=True))
for it in tqdm(range(iterations)):
    steps_left = hyp.max_steps_per_episode - environment.num_steps_taken
    valid_actions = list(environment.get_valid_actions())
    
    # Подготовка наблюдений
    observations = np.vstack([
        np.append(
            utils.get_fingerprint(act, hyp.fingerprint_length, hyp.fingerprint_radius),
            steps_left
        ) for act in valid_actions
    ])
    
    observations_tensor = torch.Tensor(observations)
    
    # Выбор действия
    action_idx = agent.get_action(observations_tensor, max(0.1, eps_threshold))
    action = valid_actions[action_idx]
    
    # Шаг среды
    result = environment.step(action)
    next_state, reward, done = result
    
    # Сохранение в буфер (с n-step)
    action_fingerprint = np.append(
        utils.get_fingerprint(action, hyp.fingerprint_length, hyp.fingerprint_radius),
        steps_left
    )

    steps_left_next = steps_left - 1 if not done else 0
    next_state_fp = np.append(
        utils.get_fingerprint(action, hyp.fingerprint_length, hyp.fingerprint_radius),
        steps_left_next
    )
    

    # Добавляем переход в буфер 
    agent.replay_buffer.add((
        action_fingerprint, 
        action_idx, 
        reward, 
        next_state_fp, 
        float(done)
    ), n_step=3, gamma=hyp.discount_factor)
    
    # Обновление модели
    if it % hyp.update_interval == 0 and len(agent.replay_buffer) > hyp.batch_size:
        loss = agent.update_params(hyp.batch_size, hyp.gamma, hyp.polyak)
        
        if TENSORBOARD_LOG and loss is not None:
            writer.add_scalar("training/loss", loss, it)
    
    # Обработка завершения эпизода
    if done:
        final_reward = reward
        
        if TENSORBOARD_LOG:
            writer.add_scalar("episode/reward", final_reward, episodes)
            writer.add_scalar("episode/epsilon", eps_threshold, episodes)
        
        # Логирование и сохранение лучшей модели
        if episodes % 10 == 0:
            print(f"Episode {episodes}, Reward: {final_reward:.2f}, Best Reward: {best_reward:.2f}, Eps: {eps_threshold:.3f}")
            mol = Chem.MolFromSmiles(environment._state)
            if mol:
                activity = predict_activity(environment._state)
                IC50 = float(activity['IC50'])
                AlogP = float(activity['AlogP'])
                qed = QED.qed(mol)

                sa = sascorer.calculateScore(mol)
                print(f"  QED: {qed:.3f}, IC50: {IC50:.3f},  AlogP: {AlogP:.3f}, SA: {sa:.3f} Molecule: {environment._state}")
                
        if final_reward > best_reward:
            torch.save(agent.dqn.state_dict(), 'best_weights.pt')
            print(f"Saved best model with reward: {final_reward:.2f}")
            best_reward = final_reward
        
        # Сброс среды и уменьшение epsilon
        environment.initialize()
        eps_threshold = max(hyp.epsilon_end, eps_threshold * hyp.gamma)
        episodes += 1

# Закрытие логгера
if TENSORBOARD_LOG:
    writer.close()

  0%|                                                                                        | 0/25000 [00:00<?, ?it/s]C:\Users\dmits\AppData\Local\Temp\ipykernel_33528\2343116160.py:134: RuntimeWarning: overflow encountered in exp
  activity_reward = 10 / (1 + np.exp(0.5 * (ic50 - 100)))
  0%|                                                                             | 30/25000 [00:10<2:51:06,  2.43it/s]

Episode 0, Reward: -1.17, Best Reward: -1000.00, Eps: 0.600
  QED: 0.199, IC50: 888.274,  AlogP: 1.389, SA: 7.299 Molecule: C=NC12OSc3cc4c(c(-n5sc(=O)n(C6c7c8cc(c(O)c76)NS8)c5=O)c31)=CC21CC=41
Saved best model with reward: -1.17


  0%|▎                                                                           | 120/25000 [00:42<2:13:03,  3.12it/s]

Saved best model with reward: 0.34


  1%|▍                                                                           | 150/25000 [00:53<2:18:15,  3.00it/s]

...

Saved best model with reward: 53.19


 39%|████████████████████████████▋                                            | 9805/25000 [1:25:07<2:32:24,  1.66it/s]

Episode 350, Reward: -0.04, Best Reward: 53.19, Eps: 0.148
  QED: 0.088, IC50: 776.094,  AlogP: 4.212, SA: 5.984 Molecule: [CH]C(c1cc(NO)cc2c3c(c4c5c(S)cc6cc5n(c4c12)C6)C(=O)NC3=O)C1SN1S


 40%|████████████████████████████▊                                           | 10025/25000 [1:27:13<2:10:18,  1.92it/s]


KeyboardInterrupt: 

In [19]:
generated_molecules = []
agent.dqn.load_state_dict(torch.load('best_weights.pt', weights_only=True))
num_molecules_to_generate = 300
agent.dqn.eval()
eps_threshold = 0.03

for it in range(num_molecules_to_generate):
    done = False
    environment.initialize()
    while not done:
        steps_left = hyp.max_steps_per_episode - environment.num_steps_taken
        valid_actions = list(environment.get_valid_actions())
    
        observations = np.vstack(
            [
                np.append(
                    utils.get_fingerprint(
                        act, hyp.fingerprint_length, hyp.fingerprint_radius
                    ),
                    steps_left,
                )
                for act in valid_actions
            ]
        ) 
    
        observations_tensor = torch.Tensor(observations)
        a = agent.get_action(observations_tensor, eps_threshold)
        action = valid_actions[a]
        result = environment.step(action)
    
        action_fingerprint = np.append(
            utils.get_fingerprint(action, hyp.fingerprint_length, hyp.fingerprint_radius),
            steps_left,
        )
    
        next_state, reward, done = result
        steps_left = hyp.max_steps_per_episode - environment.num_steps_taken
    
        next_state = utils.get_fingerprint(
            next_state, hyp.fingerprint_length, hyp.fingerprint_radius
        )  
    
        action_fingerprints = np.vstack(
            [
                np.append(
                    utils.get_fingerprint(
                        act, hyp.fingerprint_length, hyp.fingerprint_radius
                    ),
                    steps_left,
                )
                for act in environment.get_valid_actions()
            ]
        )
        if reward > 30:
            generated_molecules.append(environment._state)
            print(generated_molecules[-1])
        if len(generated_molecules) == num_molecules_to_generate:
            break
        print(reward)
    if len(generated_molecules) == num_molecules_to_generate:
        break
        
    #generated_molecules.append(environment._state)
    #print(generated_molecules[-1])

C:\Users\dmits\AppData\Local\Temp\ipykernel_33528\2343116160.py:134: RuntimeWarning: overflow encountered in exp
  activity_reward = 10 / (1 + np.exp(0.5 * (ic50 - 100)))


3.180538557836398
3.1235761272087124
3.1194069726140348
3.1194069726140343
2.6722052292129566
2.6722052292129566
2.6722052292129566
2.6722052292129566
2.6722052292129566
3.1194069726140348
3.1781723669982154
3.0489651785208194
2.7630303997671457
2.3617036636134574
2.1409490227414696
2.1409490227414696
2.1409490227414696
2.1409490227414696
-0.09749912989736224
-0.4392138642344335
-0.33564906646560433
-0.3679912968192568
-0.6402631460730884
-0.8504989798915996
-0.8504989798915996
-1.0725266236958462
-1.2688962140143956
-1.211012702794386
-0.5904397257259042
-0.8930788501412705
Cc1cc(F)ccc1C1CN(c2nc(-c3ccncc3)c(C)c(=O)n2C)CCN1.Cl
52.961503468853
3.2369253000761846
2.973251250720048
1.4470483485348973
1.1849760703858803
1.4470483485348973
1.4640956185052474
1.30503020470573
1.4640956185052474
1.6038034698381582
1.3635705579897996
0.4906699514125283
C=COC(CCNC(OO)c1nc(-c2cccnc2)no1)N1CCN(c2ccccc2CO)CC1C
49.94639857606416
3.596904743300353
1.3101051758256308
C=COC(CCNC(OO)c1nc(-c2ccc(C)nc2)n

In [21]:
list(set(generated_molecules))

['[H]C1C2C(=NO)C(CN1c1cc(OC)c(O)cc1CO)N2C(CCNC(OO)c1nc(-c2ccc3nc2C3C)no1)OC=C=S',
 'Cc1cc(CCN(N=O)C(=O)c2ccnc3[nH]c(-c4ccccc4F)nc23)ccc1NS(C)(=O)=O',
 '[H]c1c2c(O)cc(NS(C)(=O)=O)c1N(C(=O)c1ccnc3[nH]c(-c4sc5cc4-5)nc13)CC2=C',
 '[H]c1cc(CN(C=Cc2ccc(NS(C)(=O)=O)c(CO)c2)N=O)c2nc(-c3c(F)c(O)c4c(O)c3-4)[nH]c2n1',
 '[H]c1ccc(-c2nc3c(C(=O)N(CCc4ccc(NS(C)(=O)=O)c(C)c4)N=O)cc(C)nc3[nH]2)c(F)c1O',
 '[H]Cc1c2c(CC(=O)N(S)C(=O)c3ccc(O)c4[nH]c(-c5c6cc(F)c(C=N)c5S6)nc34)c-2c(O)c1N(OO)S(=O)(=O)c1ccc(CO)cc1',
 '[H]c1cc(O)c(-c2ccc3c(NC(=O)C4C=C4)n[nH]c3n2)cc1O',
 'CS(=O)(=O)Nc1ccc(C=CN(Cc2ccnc3[nH]c(-c4c(F)c(O)c5c(O)c4-5)nc23)N=O)cc1CO',
 '[H]c1cc(S(=O)(=O)N2C#CN(C)CC2)c(-c2nc(C(=O)Nc3cccnc3)c3nc2N3)cc1C',
 '[H]NN1c2nc3c(C(=O)n4c5cc(N6CCNC(=O)C6)c4cn5)nc2-c2c(ccc(CO)c2F)N31',
 '[H]c1cc(-c2cnccc2OC)c(S)c2c(C(=O)N(O)C(=O)C3CCN(C(=S)c4ccc(C(=O)O)o4)CC3)n[nH]c12',
 '[H]c1c(O)ccc(O)c1-c1ccc2c(NC(=S)C3=C=C3)n[nH]c2n1',
 '[H]N1CCN(c2cc3ncc2n3C(=O)c2nc(-c3ccccc3F)cnc2N)CC1=O',
 'C=C1C2=C(C=CN1c1nccc(-c3cnn4nc(OC

In [22]:
pd.DataFrame(list(set(generated_molecules))).to_csv("generated_mols.csv")

In [ ]:
from rdkit import Chem

valid_smiles = []
for smi in generated_molecules:
    mol = Chem.MolFromSmiles(smi)
    if mol is not None:
        valid_smiles.append(smi)

print(f"Сгенерировано валидных молекул: {len(valid_smiles)}/{len(generated_molecules)}")